In [35]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from math import sqrt

In [158]:
df = pd.read_csv('./room_information.csv')

In [159]:
df.head()

,Area,Building,Floor / Room,Room type,Seats,Seating,Floor area,Floor shape,07_00_mo,07_00_tu,...,23_45_we,23_45_th,23_45_fr,23_45_sa,24_00_mo,24_00_tu,24_00_we,24_00_th,24_00_fr,24_00_sa
0,Zürich Zentrum,CAB,G 11,Lecture hall,190.0,fixed,212.95,graded,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Zürich Zentrum,CAB,G 51,Lecture hall,90.0,fixed,101.06,graded,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Zürich Zentrum,CAB,G 52,Seminars / Courses,40.0,variable,67.27,flat,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Zürich Zentrum,CAB,G 56,Seminars / Courses,40.0,variable,67.32,flat,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Zürich Zentrum,CAB,G 57,Seminars / Courses,24.0,variable,44.14,flat,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [160]:
df['Area'].value_counts()

Zürich Zentrum        216
Zürich Hönggerberg    153
Lindau Eschikon         3
Schwerzenbach           3
Zürich Universität      2
Zürich Oerlikon         1
Name: Area, dtype: int64

In [161]:
df['Seats'].value_counts()

30.0     33
40.0     28
16.0     18
24.0     17
32.0     15
         ..
153.0     1
502.0     1
47.0      1
96.0      1
104.0     1
Name: Seats, Length: 80, dtype: int64

In [155]:
print(df.iloc[0])
print(df.iloc[0]['09_45_tu'])

Area            Zürich Zentrum
Building                   CAB
Floor / Room              G 11
Room type         Lecture hall
Seats                    190.0
                     ...      
22_00_we                   NaN
22_00_th                   NaN
22_00_fr                   NaN
22_00_sa                   NaN
scores                   33.65
Name: 0, Length: 375, dtype: object
Advanced Dynamics


In [162]:
df['Room type'].value_counts()

Seminars / Courses       144
Lecture hall              75
Meeting room              50
Laboratory internship     47
Computer                  30
Draw                       8
Technology laboratory      5
Training room              5
Exercises                  4
Multipurpose room          4
Microscopy                 2
Canteen kitchen            1
Photo lab / Darkroom       1
Office                     1
Isotope laboratory         1
Name: Room type, dtype: int64

In [137]:
def get_room_type_score(df_series):
    room_types = {'Seminars / Courses': 100, 'Lecture hall': 30, 'Meeting room': 100, 'Laboratory internship': 10, 'Computer': 70, 'Draw':50, 'Exercises': 20}

    if df_series['Room type'] in room_types:
        return room_types[df_series['Room type']]

    else: return 0

In [78]:
def get_timestamp(available_later=False, time:datetime = None) -> str:
    if not time:
        time = datetime.now()

    rounded_time = time - timedelta(minutes=time.minute % 15)

    hours = rounded_time.strftime("%H")
    minutes = rounded_time.strftime("%M")
    day = rounded_time.strftime("%A")

    days_translation = {'Monday':'mo', 'Tuesday':'tu', 'Wednesday':'we', 'Thursday':'th', 'Friday':'fr', 'Saturday':'sa', 'Sunday':'su'}

    if available_later:
        timestamp = str(int(hours)+1).zfill(2) + '_{}_{}'.format(minutes, days_translation[day])
    else:
        timestamp = str(hours).zfill(2) + '_{}_{}'.format(minutes, days_translation[day])


    return timestamp


In [103]:
def is_available(df_series, timestamp) -> bool:
    
    return pd.isna(df_series[timestamp])



#is_available(df_series=df.iloc[0], timestamp = get_timestamp(available_later=False, time = datetime(2023, 3, 8, 9, 12, 0, 0)))

In [102]:
def distance_to_room(df_series, location):
    # Build 2d Map stored on Notability
    areas_2d = {'Zürich Zentrum': (1.85, 0), 'Zürich Hönggerberg': (0, 2.25), 'Lindau Eschikon': (8.15, 5), 'Schwerzenbach': (7, 0.55), 'Zürich Universität': (2, 1.5), 'Zürich Oerlikon': (1.5, 3)}

    room_vec = areas_2d[df_series['Area']]
    current_vec = areas_2d[location]

    distance = sqrt((room_vec[0] - current_vec[0])**2 + (room_vec[1] - current_vec[1])**2)

    return distance


# location_ = 'Lindau Eschikon'
# print(max(0, 100 + distance_to_room(df_series=df.iloc[0], location=location_) ** 2 - 21 * distance_to_room(df_series=df.iloc[0], location=location_)))

In [101]:
def has_been_used(df_series, timestamp) -> bool:
    '''
    Returns if room has been used this day, increasing its chances of being open
    '''

    # Only check for previous hours
    df_series = df_series.iloc[8:df_series.index.get_loc(timestamp)+1]

    # Clean dataset from remaining days of the week
    df_series = df_series.loc[df_series.index.str.contains(timestamp[-2:])]

    return pd.notna(df_series).any()

#has_been_used(df_series=df.iloc[0], timestamp = get_timestamp(available_later=False, time = datetime(2023, 3, 7, 7, 50, 0, 0)))

In [127]:
def will_be_used_in(df_series, timestamp):
    '''
    Returns time room will be free and corresponding score
    '''


    # Only check for upcoming hours
    df_series = df_series.iloc[df_series.index.get_loc(timestamp):]

    # Clean dataset from remaining days of the week
    df_series = df_series.loc[df_series.index.str.contains(timestamp[-2:])]

    for nb_quarters, val in enumerate(df_series):
        if not pd.isna(val):
            break

    remaining_time_room_free = nb_quarters * 15
    remaining_time_room_free = str(remaining_time_room_free // 60) + str(remaining_time_room_free % 60).zfill(2)

    score =  min(100, 0.5 * nb_quarters * 15 + 0.001 * (nb_quarters * 15) ** 2)
    
    return remaining_time_room_free, score



#will_be_used_in(df_series=df.iloc[0], timestamp = get_timestamp(available_later=False, time = datetime(2023, 3, 8, 7, 30, 0, 0)))


('200', 74.4)

In [180]:
def value_function(df_series, location, time=None, available_later=False):
    '''
    Policy from most important to least:
    1. Availability - is room available?
     -> accounts for 51% of score
    2. Location - how close am I to the room?
     -> accounts for 15% of score
    3. Has been used that day? - Rooms in use more likely to be open
     -> accounts for 11% of score
    4. Room type - Seminar rooms are superior to Lecture hall for studying
     -> accounts for 9% of score
    5. How long available - how long may I stay?
     -> accounts for 9% of score
    6. Seating - Larger rooms attract more people
     -> accounts for 5% of score
    '''

    timestamp = get_timestamp(available_later=available_later, time=time)

    ### NOTE: REDO evaluation for upcoming hour and and tell user

    # Check Availability
    if is_available(df_series=df_series, timestamp=timestamp):
        availability_score = 100
    else: availability_score = 0

    # 6 available locations
    # min dist: 0.0, max dist: 8.601453365565613
    distance = distance_to_room(df_series=df_series, location=location)
    location_score = max(0, 100 + distance ** 2 - 21 * distance)

    # Check if room has been used
    has_been_used_score = 100*has_been_used(df_series=df_series, timestamp=timestamp)

    # Check room type:
    room_type_score = get_room_type_score(df_series=df_series)

    # Check how long room will be available
    # Index 1 to get the score
    future_availability_score = will_be_used_in(df_series=df_series, timestamp=timestamp)[1]

    # Seating
    # Rooms above 90 are not likely to be recommended
    if not pd.isna(df_series['Seats']):
        seating_score = min(3, int(df_series['Seats']) % 30) * 33
    else: seating_score = 0


    scores = [availability_score, location_score, has_been_used_score, room_type_score, future_availability_score, seating_score]
    scores_weights = [0.51, 0.15, 0.11, 0.09, 0.09, 0.05]


    score = sum([score * weight for score, weight in zip(scores, scores_weights)])

    return score

# value_function(df_series=df.iloc[0], location = 'Lindau Eschikon', available_later=True, time = datetime(2023, 3, 8, 8, 30, 0, 0))


In [164]:
df['scores'] = df.apply(lambda row: value_function(df_series= row, location='Zürich Zentrum', time = datetime(2023, 3, 8, 9, 30, 0, 0)), axis=1)

In [181]:
df['scores_later'] = df.apply(lambda row: value_function(df_series= row, location='Zürich Zentrum', time = datetime(2023, 3, 8, 9, 30, 0, 0), available_later=True), axis=1)

10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
10_30_we
1

In [173]:
print(df['scores'].value_counts()[max(df['scores'])])
print(df['scores_later'].value_counts()[max(df['scores_later'])])

7
1


In [171]:
max_rows = df.loc[df['scores'] == df['scores'].max()]
max_rows.head(10)

,Area,Building,Floor / Room,Room type,Seats,Seating,Floor area,Floor shape,07_00_mo,07_00_tu,...,23_45_th,23_45_fr,23_45_sa,24_00_mo,24_00_tu,24_00_we,24_00_th,24_00_fr,24_00_sa,scores
23,Zürich Zentrum,CLA,D 17,Meeting room,16.0,variable,35.64,flat,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.95
147,Zürich Zentrum,HG,D 3.3,Seminars / Courses,28.0,variable,58.26,flat,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.95
150,Zürich Zentrum,HG,D 5.3,Seminars / Courses,28.0,variable,58.57,flat,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.95
187,Zürich Zentrum,HG,F 33.5,Meeting room,36.0,variable,61.42,flat,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.95
188,Zürich Zentrum,HG,F 50.3,Meeting room,17.0,variable,56.35,flat,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.95
308,Zürich Zentrum,LEE,E 126,Meeting room,20.0,variable,47.74,flat,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.95
329,Zürich Zentrum,LFW,E 13,Seminars / Courses,40.0,variable,95.97,flat,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.95


In [184]:
max_rows = df.loc[df['scores_later'] == df['scores_later'].max()]
print(max_rows['10_30_we'])
max_rows.head(10)

335    NaN
Name: 10_30_we, dtype: object


,Area,Building,Floor / Room,Room type,Seats,Seating,Floor area,Floor shape,07_00_mo,07_00_tu,...,23_45_fr,23_45_sa,24_00_mo,24_00_tu,24_00_we,24_00_th,24_00_fr,24_00_sa,scores,scores_later
335,Zürich Zentrum,ML,E 13,Meeting room,21.0,variable,61.12,flat,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.95,99.725


In [183]:
is_available(max_rows, timestamp='10_30_we')

335    True
Name: 10_30_we, dtype: bool